In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
print("Setup complete")

In [ ]:
# Generate synthetic time series
np.random.seed(42)
t = np.arange(0, 100, 0.1)
trend = 0.1 * t
seasonal = 10 * np.sin(2 * np.pi * t / 10)
noise = np.random.normal(0, 1, len(t))
ts = trend + seasonal + noise

df = pd.DataFrame({'time': range(len(ts)), 'value': ts})
print(f"Time series shape: {df.shape}")

In [ ]:
# Plot time series
plt.figure(figsize=(12, 5))
plt.plot(df['time'], df['value'], alpha=0.7)
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series Data')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Trend decomposition
from pandas.plotting import seasonal_decompose
decomposition = seasonal_decompose(pd.Series(ts), model='additive', period=100)
fig, axes = plt.subplots(4, 1, figsize=(12, 8))
axes[0].plot(decomposition.observed)
axes[0].set_ylabel('Observed')
axes[1].plot(decomposition.trend)
axes[1].set_ylabel('Trend')
axes[2].plot(decomposition.seasonal)
axes[2].set_ylabel('Seasonal')
axes[3].plot(decomposition.resid)
axes[3].set_ylabel('Residual')
plt.tight_layout()
plt.show()

In [ ]:
# Differencing
diff = np.diff(ts)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(ts)
plt.title('Original')
plt.subplot(1, 2, 2)
plt.plot(diff)
plt.title('First Difference')
plt.tight_layout()
plt.show()

In [ ]:
# Moving average
window = 20
ma = pd.Series(ts).rolling(window=window).mean()
plt.figure(figsize=(12, 5))
plt.plot(ts, alpha=0.5, label='Original')
plt.plot(ma, label=f'MA-{window}', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Moving Average')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Exponential smoothing
from scipy.interpolate import UnivariateSpline
alpha = 0.3
es = [ts[0]]
for t in ts[1:]:
    es.append(alpha * t + (1 - alpha) * es[-1])

plt.figure(figsize=(12, 5))
plt.plot(ts, alpha=0.5, label='Original')
plt.plot(es, label=f'Exponential Smoothing (α={alpha})', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Exponential Smoothing')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Autoregressive model (simple)
def simple_ar(y, lag=5, test_size=0.2):
    n = len(y)
    train_size = int(n * (1 - test_size))
    train, test = y[:train_size], y[train_size:]
    
    predictions = []
    for t in range(len(test)):
        if t < lag:
            pred = np.mean(train[-lag:])
        else:
            pred = np.mean(predictions[-lag:])
        predictions.append(pred)
    
    mae = mean_absolute_error(test[lag:], predictions[lag:])
    rmse = np.sqrt(mean_squared_error(test[lag:], predictions[lag:]))
    return predictions, mae, rmse

predictions, mae, rmse = simple_ar(ts, lag=5)
print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")

In [ ]:
# Forecast
test_start = int(len(ts) * 0.8)
plt.figure(figsize=(12, 5))
plt.plot(ts, label='Original')
plt.plot(range(test_start, len(predictions) + test_start), predictions[:len(ts)-test_start], label='Forecast')
plt.axvline(test_start, color='red', linestyle='--', alpha=0.5)
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series Forecast')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
test_results = []
test1 = len(decomposition.trend) > 0
test_results.append(("Test 1: Decomposition", test1))
test2 = len(diff) == len(ts) - 1
test_results.append(("Test 2: Differencing", test2))
test3 = len(ma) == len(ts)
test_results.append(("Test 3: Moving Average", test3))
test4 = len(es) == len(ts)
test_results.append(("Test 4: Exponential Smoothing", test4))
test5 = mae > 0 and rmse > 0
test_results.append(("Test 5: AR Model", test5))
passed = sum(1 for _, r in test_results if r)
print(f"\nPASSED: {passed}/{len(test_results)}")